<a href="https://colab.research.google.com/github/Waltberry/Machine_Learning_Training_Repo/blob/main/Custom_Python_Model_Main_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Custom Model For Concrete Strength

This is a sample notebook we would use to build a custom model for a specific dataset. We have chosen to use a Concrete Strength dataset that describes the properties of concrete for thie purpose of demo during this workshop. 

If you'd like to write your own custom transformations, you can use this notebook with placeholders for you to fill in.
https://colab.research.google.com/drive/1eCx0Rhz6IxVR_QD0ikS4Npz6lO8sxzca

#### Extra libraries required in Colab

In [ ]:
!pip install s3fs

#### Standard Libraries

In [ ]:
import pandas as pd

### Loading and inspecting the data

We start by loading the dataset we are going to work with.

In [ ]:
concrete_df = pd.read_csv('s3://abacusai.exampledatasets/predicting/concrete_measurements.csv')
concrete_df.describe()

,cement,slag,flyash,water,superplasticizer,coarseaggregate,fineaggregate,age,csMPa
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


#### Custom Data Transform

We are going to transform the dataset so that flyash is no longer a feature but instead all the other values are transformed according to whether they have flyash > 0 or not.

The example is not entirely realistic and it is certainly feasible to achieve the same result using SQL. However, the point is to illustrate that you are free to transform the dataset using the full functionality of python and its data frameworks. Here we are using pandas but you can use a wide range of standard python libraries to manipulate the data. Additionally, you can bundle resources with your code, for example small maps or tables, that can be accessed by your function to implement the transform.

In [ ]:
def transform_concrete(concrete_dataset):
  import pandas as pd
  feature_df = concrete_dataset.drop(['flyash'], axis=1)
  no_flyash = feature_df[concrete_dataset.flyash == 0.0]
  flyash = feature_df[concrete_dataset.flyash > 0.0]
  return pd.concat([no_flyash - no_flyash.assign(age=0).mean(), flyash - flyash.assign(age=0).mean()])

transformed_concrete_df = transform_concrete(concrete_df)
transformed_concrete_df.describe()

,cement,slag,water,superplasticizer,coarseaggregate,fineaggregate,age,csMPa
count,1.030000e+03,1.030000e+03,1.030000e+03,1.030000e+03,1.030000e+03,1.030000e+03,1030.000000,1.030000e+03
mean,-9.348811e-14,9.875854e-14,1.342995e-13,-4.492631e-15,-5.190963e-13,4.466899e-13,45.662136,-2.355829e-15
std,9.799162e+01,8.127146e+01,2.061237e+01,5.481593e+00,7.775244e+01,7.959387e+01,63.169912,1.667246e+01
min,-2.120378e+02,-1.001102e+02,-6.001678e+01,-8.826078e+00,-1.723574e+02,-1.722265e+02,1.000000,-3.444178e+01
25%,-7.367220e+01,-4.191875e+01,-1.340776e+01,-4.055654e+00,-4.135742e+01,-4.284638e+01,7.000000,-1.231678e+01
50%,-4.037809e+00,-2.191875e+01,-3.077586e-01,-1.826078e+00,-5.357420e+00,-1.853004e+00,28.000000,-9.444612e-01
75%,5.700280e+01,6.668975e+01,1.138322e+01,2.748922e+00,5.701595e+01,5.729700e+01,56.000000,1.040121e+01
max,2.639278e+02,2.592898e+02,7.159224e+01,2.814435e+01,1.716426e+02,2.277470e+02,365.000000,4.582822e+01


### Custom Model

#### Requires catboost

In [ ]:
!pip install catboost

### Train Function

To illustrate that the training can be customized arbitrarily we will train a composite model that depending on the age of the concrete uses a linear model on quantized features or a GBDT trained on raw inputs. It is effectively a decision tree model on the age feature where one side of the decision tree delegates to a linear model and the other side to a boosted tree model.

In [ ]:
def train(training_dataset):
  # set the seed for reproduceable results
  import numpy as np
  np.random.seed(5)

  X = training_dataset.drop(['csMPa'], axis=1)
  y = training_dataset.csMPa
  recent = training_dataset.age < 10
  from sklearn.preprocessing import QuantileTransformer
  from sklearn.linear_model import LinearRegression
  qt = QuantileTransformer(n_quantiles=20)
  recent_model = LinearRegression()
  _ = recent_model.fit(qt.fit_transform(X[recent]), y[recent])
  print(f'Linear model R^2 = {recent_model.score(qt.transform(X[recent]), y[recent])}')

  from catboost import Pool, CatBoostRegressor
  train_pool = Pool(X[~recent], y[~recent])
  older_model = CatBoostRegressor(iterations=5, depth=10, loss_function='RMSE')
  _ = older_model.fit(train_pool)
  metrics = older_model.eval_metrics(train_pool, ['RMSE'])
  old_r2 = 1 - metrics['RMSE'][-1]**2 / y[~recent].var()
  print(f'Catboost model R^2 = {old_r2}')

  return (X.columns, qt, recent_model, older_model)

local_model = train(transformed_concrete_df)

Linear model R^2 = -59474.80409065779
Learning rate set to 0.5
0:	learn: 12.7627412	total: 46.6ms	remaining: 187ms
1:	learn: 11.5585084	total: 47.5ms	remaining: 71.3ms
2:	learn: 10.3223491	total: 48.2ms	remaining: 32.2ms
3:	learn: 9.3247540	total: 52.1ms	remaining: 13ms
4:	learn: 8.5430952	total: 52.7ms	remaining: 0us
Catboost model R^2 = 0.6814947748102853


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:439: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


### Prediction Function

In the example we are building the model is a composite model built on two partitions of the data so the prediction function needs to dispatch the input to the right model based on one of the input features.

In [ ]:
def predict(model, query):
  columns, qt, recent_model, older_model = model
  import pandas as pd
  X = pd.DataFrame({c: [query[c]] for c in columns})
  if X.age[0] < 10:
    y = recent_model.predict(qt.transform(X))[0]
  else:
    y = older_model.predict(X.values.reshape(-1))
  return {'csMPa': y}

for _, r in transformed_concrete_df[transformed_concrete_df.age < 10][:5].iterrows():
  print(predict(local_model, r.to_dict()), r['csMPa'])

for _, r in transformed_concrete_df[transformed_concrete_df.age > 10][:5].iterrows():
  print(predict(local_model, r.to_dict()), r['csMPa'])

{'csMPa': -31.75412474980192} -28.711784452296826
{'csMPa': -5.324797742032455} 1.8282155477031736
{'csMPa': -4.377726654712578} -1.6917844522968295
{'csMPa': -23.147157848108026} -21.721784452296827
{'csMPa': -16.712019233341156} -10.511784452296826
{'csMPa': 19.340295162698652} 43.21821554770317
{'csMPa': 19.340295162698652} 25.118215547703173
{'csMPa': 10.376285866501192} 3.4982155477031753
{'csMPa': 10.376285866501192} 4.278215547703169
{'csMPa': -2.273645085750303} 7.528215547703169




## Integrate with Abacus.AI

Many data science projects stop at this point. Actually, most don't even clearly define the prediction operation that will be used in production applications. To actually leverage this model in production generally requires quite a bit more work:
- Storing the model so that it is available in various production workflows
- Hosting the model in a scalable manner so that it can be used for online predictions
- Support for evaluating the model against large batches of new data
- Monitoring the model to ensure its inputs and predictions have not shifted significantly

Beyond this specific features to support model usage there is the significant task of reliably keeping the model up-to-data as new data arrives. This involves a workflow of operations starting with the refresh of the input datasets through pushing the models to serving infrastructure.

Real world machine learning applications require performing all these operations reliably.

### Use Abacus.AI for all this and more

- [Sign up](https://abacus.ai/app/signup?signupToken=python_models) for an Abacus.AI Account
- Once your account is created, navigate to the [API Keys Dashboard](https://abacus.ai/app/profile/apikey) and generate an API key to authenticate your ApiClient

# Abacus.AI Integration Notebook

https://colab.research.google.com/drive/1AVvPE5Ue89l5n8Ed9eqdjAV5NQHMEMyl
